## Natural Language Processing with RNNs and Attention

> ### Generating Shakespearean Text Using a Character RNN

In [ ]:
import tensorflow as tf
from IPython.display import clear_output
tf.multiply(2, 3)
clear_output()

>> #### Creating the Training Dataset

>>> Download dataset using get_file() from keras.utils

In [ ]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

>>> Encode character as an integer

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [ ]:
tokenizer.texts_to_sequences(['First'])

In [ ]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

In [ ]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [ ]:
# Let’s encode the full text so each character is represented by its ID 
# (we subtract 1 to get IDs from 0 to 38, rather than from 1 to 39):

import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

>> #### How to Split a Sequential Dataset

In [ ]:
training_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:training_size])
training_size

>> #### Chopping the Sequential Dataset into Multiple Windows

In [ ]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

# The window() method creates a dataset that contains windows, each of which is also
# represented as a dataset. It’s a nested dataset, analogous to a list of lists.

In [ ]:
for i in dataset.take(5):
    for j in i.take(5):
        print(j.numpy(), end=' ')

>>> Convert to flat dataset

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

>>> Since Gradient Descent works best when the instances in the training set are independent and identically distributed we need to shuffle these windows.

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

>>> Convert to embedding

In [ ]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

>> #### Building and Training the Char - RNN Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                        dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True,
                        dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                            activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

>> #### Using the Char - RNN Model

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

>>> Generating Fake Shakespearean Text

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
complete_text("t", temperature=0.2)

In [ ]:
complete_text("t", temperature=1)

In [ ]:
complete_text("w", temperature=0.2)

>> #### Stateful RNN

>>> Build data for stateful RNN (not same as stateless RNN)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:dataset_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

>>> Build Stateful RNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                        dropout=0.2, recurrent_dropout=0.2,
                        batch_input_shape=[batch_size, None, max_id]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True,
                        dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id,
                                                            activation="softmax"))
])

In [ ]:
# At the end of each epoch, we need to reset the states before we go back 
# to the beginning of the text. For this, we can use a small callback:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10, callbacks=[ResetStatesCallback()])